In [ ]:
# f001

In [61]:
penncli = PennsieveClient()

In [45]:
max([v["content"]["updatedAt"] for v in penncli.get_dataset(did)["children"]])

'2024-06-29T22:46:50.374403Z'

In [62]:
# penncli = PennsieveClient()
did = "N:dataset:aa43eda8-b29a-4c25-9840-ecbd57598afc"
manifest = penncli.get_dataset_manifest(did)
manifest


def pull_files(id_or_name: str, pattern: str | None = None) -> list[str]:
    """Export a dataset to a directory

    Parameters
    ----------
    id_or_name : str
        Dataset ID or name
    output_dir : Path | str
        Output directory
    verbose : bool, optional
        Prints filename being downloaded, by default True
    """
    files = []
    url = "https://api.pennsieve.io/packages/download-manifest"
    # output_dir = Path(output_dir) / id_or_name
    # Pull dataset for root children IDs
    dataset = penncli.get_dataset(id_or_name)
    for child in dataset["children"]:
        payload = {"nodeIds": [child["content"]["nodeId"]]}
        # Pull tree for 1 child at a time since prebuilt s3 links only last a couple hours
        # If all children are pulled at once, the links will expire before all files are downloaded if over 200GBs
        # resp = penncli._post(url, payload=payload)
        # try:
        #     manifest = resp.json()
        # except:
        #     print(payload)
        #     print(resp.text)
        #     continue
        manifest = penncli.get_child_manifest(
            node_id=child["content"]["nodeId"]
        )
        for filemeta in manifest["data"]:
            # filemeta[""] = manifest["datasetId"]
            if Path(filemeta["fileName"]).suffix in [".csv"]:
                files.append(filemeta)
    return dataset, files


dataset, files = pull_files(did)
fascicle_files = [v for v in files if "fascicles" in v["fileName"].lower()]
file = fascicle_files[-1]
file

{'nodeId': 'N:package:cc6c1783-4eb4-4a0a-b723-247827ebeb4e',
 'fileName': 'naveen_paper_B733_T7L_A_LEVEL_3_fascicles.csv',
 'packageName': 'naveen_paper_B733_T7L_A_LEVEL_3_fascicles.csv',
 'path': ['derivative', 'sam-l', 'sam-l-seg-t7', 'sam-l-seg-t7-LEVEL_3'],
 'url': 'https://prd-sparc-storage-use1.s3.amazonaws.com/O367/D3090/be5e20cb-86e2-41ea-a72f-e29d61354c96/cc6c1783-4eb4-4a0a-b723-247827ebeb4e?response-content-disposition=attachment%3B%20filename%3D%22naveen_paper_B733_T7L_A_LEVEL_3_fascicles.csv%22&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEH8aCXVzLWVhc3QtMSJHMEUCIAPch%2BauQrPQv21R0RXeIldFPBezXJBak%2FnCmQ%2F317bTAiEA%2Fk%2B653R0GVTYYy3RlaE5nnPR3oEjUkHPZ%2FzfzNWZFCAqigQIp%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARADGgw3NDA0NjMzMzcxNzciDN3ESJWEi1SigEZhZireA4etY0qpkpsnV%2FLZKvXvfmPzVQZ%2F96LEJUqCnAEttICbb%2BmKLcIqKm4AvjziRHltIQmsmeFHLazhcNzCZ%2Bhgz1o%2BuhDFeFWimy3UNxwKGXn6g0sc5sCMytJBXKKL8Omq5tSDpVVn7quipiIlZbM1S0nLed6LNZ4ScNHhSA%2Fues8byTAHTBZXV3OUML9AFrAcggwvoxdwCM28G4lS%2B0vY4vM0%2BT84%2BKh

In [63]:
import pandas as pd

df = pd.read_csv(file["url"])
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])
df.head()

,fascicle,area,longest_diameter,shortest_diameter,eff_diam,c_estimate_nav,c_estimate_nf,nfibers_w_c_estimate_nav,nfibers_w_c_estimate_nf,nfibers_all,...,epi_dist,epi_dist_inv,nerve_based_area,nerve_based_perimeter,nerve_based_eff_diam,perinerium_vertices,perinerium_vertices_px,nerve_based_shortest_diameter,hull_contrs,hull_contr_areas
0,1,601840.629341,907.000466,846.340808,875.378369,0,16502,4613,21115,21115,...,515.931,0.000,477860.198,4084.885,780.020,[[3127.5 113.4 ]\n [3160.35 113.4 ]\n [3192...,[[20850 756]\n [21069 756]\n [21280 826]...,630.464,[ 0. 5. 10. 15. 20. 25. 30. 35. 40. ...,[3.38965189e+06 3.29847812e+06 3.20848098e+06 ...
1,2,151657.783648,566.334306,329.680645,439.427682,0,1300,1548,2848,2848,...,274.845,241.087,163417.736,1544.309,456.147,[[1488.15 480.75]\n [1504.8 485.55]\n [1521...,[[ 9921 3205]\n [10032 3237]\n [10143 3262]...,350.905,[ 0. 5. 10. 15. 20. 25. 30. 35. 40. ...,[1.08945157e+06 1.03852465e+06 9.88700989e+05 ...
2,3,42956.807122,253.176413,208.473851,233.868137,0,239,289,528,528,...,239.555,276.376,51265.699,894.344,255.487,[[605.55 361.5 ]\n [614.85 361.5 ]\n [624.3 3...,[[4037 2410]\n [4099 2410]\n [4162 2410]\n [42...,215.234,[ 0. 5. 10. 15. 20. 25. 30. 35. 40. ...,[3.41771325e+05 3.12657208e+05 2.84935875e+05 ...
3,4,19745.681094,177.272006,131.860742,158.559080,0,108,120,228,228,...,394.411,121.521,24790.376,657.547,177.663,[[728.1 470.85]\n [734.7 469.2 ]\n [741. 4...,[[4854 3139]\n [4898 3128]\n [4940 3123]\n [49...,138.983,[ 0. 5. 10. 15. 20. 25. 30. 35. 40. 45. 50. 5...,[165269.175 144192.08666671 124780.433718...
4,5,72513.774721,370.254924,237.445074,303.854250,0,630,695,1325,1325,...,432.613,83.318,81804.589,1104.140,322.733,[[1107. 441.45]\n [1115.55 446.4 ]\n [1119...,[[7380 2943]\n [7437 2976]\n [7462 3032]\n [75...,250.218,[ 0. 5. 10. 15. 20. 25. 30. 35. 40. ...,[545363.925 509233.56034859 474385.219163...


In [11]:
from quantdb.config import auth
from sqlalchemy import create_engine
from quantdb.utils import dbUri, isoformat, log
from sqlalchemy.sql import text as sql_text
from sqlalchemy.orm import Session

# pull in the db connection info
dbkwargs = {
    k: auth.get(f"db-{k}") for k in ("user", "host", "port", "database")
}  # TODO integrate with cli options
# custom user variable needed
dbkwargs["dbuser"] = dbkwargs.pop("user")
# create connection env with DB
engine = create_engine(dbUri(**dbkwargs))
# bool: echo me
engine.echo = True
# use connection env as unique session
session = Session(engine)


class Queries:
    def __init__(self, session):
        self.session = session

    def address_from_fadd_type_fadd(self, fadd_type, fadd):
        # FIXME multi etc.
        res = [
            i
            for i, in self.session.execute(
                sql_text(
                    "select * from address_from_fadd_type_fadd(:fadd_type, :fadd)"
                ),
                dict(fadd_type=fadd_type, fadd=fadd),
            )
        ]
        if res:
            return res[0]

    def desc_inst_from_label(self, label):
        # FIXME multi etc.
        res = [
            i
            for i, in self.session.execute(
                sql_text("select * from desc_inst_from_label(:label)"),
                dict(label=label),
            )
        ]
        if res:
            return res[0]

    def desc_quant_from_label(self, label):
        # FIXME multi etc.
        res = [
            i
            for i, in self.session.execute(
                sql_text("select * from desc_quant_from_label(:label)"),
                dict(label=label),
            )
        ]
        if res:
            return res[0]

    def desc_cat_from_label_domain_label(self, label, domain_label):
        # FIXME multi etc.
        res = [
            i
            for i, in self.session.execute(
                sql_text(
                    "select * from desc_cat_from_label_domain_label(:label, :domain_label)"
                ),
                dict(label=label, domain_label=domain_label),
            )
        ]
        if res:
            return res[0]

    def cterm_from_label(self, label):
        # FIXME multi etc.
        res = [
            i
            for i, in self.session.execute(
                sql_text("select * from cterm_from_label(:label)"),
                dict(label=label),
            )
        ]
        if res:
            return res[0]

    def insts_from_dataset_ids(self, dataset, ids):
        return list(
            self.session.execute(
                sql_text(
                    "select * from insts_from_dataset_ids(:dataset, :ids)"
                ),
                dict(dataset=dataset, ids=ids),
            )
        )


class InternalIds:
    def set_desc_quant_from_label(self, label: str, desctriptor: str):
        self.__setattr__(label, desctriptor)

    def __init__(self, queries) -> None:

        q = queries
        self._q = queries

        self.addr_suid = q.address_from_fadd_type_fadd(
            "tabular-header", "id_sub"
        )
        self.addr_said = q.address_from_fadd_type_fadd(
            "tabular-header", "id_sam"
        )
        self.addr_spec = q.address_from_fadd_type_fadd(
            "tabular-header", "species"
        )
        self.addr_saty = q.address_from_fadd_type_fadd(
            "tabular-header", "sample_type"
        )
        self.addr_faid = q.address_from_fadd_type_fadd(
            "tabular-header", "fascicle"
        )  # for REVA ft

        self.addr_tmod = q.address_from_fadd_type_fadd(
            "tabular-header", "modality"
        )
        # addr_trai = address_from_fadd_type_fadd('tabular-header', 'raw_anat_index')
        # addr_tnai = address_from_fadd_type_fadd('tabular-header', 'norm_anat_index')
        # addr_context = address_from_fadd_type_fadd('context', '#/path-metadata/{index of match remote_id}/dataset_relative_path')  # XXX this doesn't do what we want, I think what we really would want in these contexts are objects_internal that reference the file system state for a given updated snapshot, that is the real "object" that corresponds to the path-metadata.json that we are working from

        # addr_jpmod = address_from_fadd_type_fadd('json-path-with-types', '#/#int/modality')
        # addr_jprai = address_from_fadd_type_fadd('json-path-with-types', '#/#int/anat_index')
        # addr_jpnai = address_from_fadd_type_fadd('json-path-with-types', '#/#int/norm_anat_index')

        self.addr_jpdrp = q.address_from_fadd_type_fadd(
            "json-path-with-types",
            "#/path-metadata/data/#int/dataset_relative_path",
        )

        # XXX these are more accurate if opaque
        self.addr_jpmod = q.address_from_fadd_type_fadd(
            "json-path-with-types",
            "#/path-metadata/data/#int/dataset_relative_path#derive-modality",
        )
        # addr_jprai = address_from_fadd_type_fadd('json-path-with-types', '#/path-metadata/data/#int/dataset_relative_path#derive-raw-anat-index')
        self.addr_jpnai = q.address_from_fadd_type_fadd(
            "json-path-with-types",
            "#/path-metadata/data/#int/dataset_relative_path#derive-norm-anat-index-v1",
        )
        self.addr_jpnain = q.address_from_fadd_type_fadd(
            "json-path-with-types",
            "#/path-metadata/data/#int/dataset_relative_path#derive-norm-anat-index-v1-min",
        )
        self.addr_jpnaix = q.address_from_fadd_type_fadd(
            "json-path-with-types",
            "#/path-metadata/data/#int/dataset_relative_path#derive-norm-anat-index-v1-max",
        )
        self.addr_jpsuid = q.address_from_fadd_type_fadd(
            "json-path-with-types",
            "#/path-metadata/data/#int/dataset_relative_path#derive-subject-id",
        )
        self.addr_jpsaid = q.address_from_fadd_type_fadd(
            "json-path-with-types",
            "#/path-metadata/data/#int/dataset_relative_path#derive-sample-id",
        )

        self.addr_jpspec = q.address_from_fadd_type_fadd(
            "json-path-with-types", "#/local/tom-made-it-up/species"
        )
        self.addr_jpsaty = q.address_from_fadd_type_fadd(
            "json-path-with-types", "#/local/tom-made-it-up/sample_type"
        )

        # future version when we actually have the metadata files
        # addr_jpmod = address_from_fadd_type_fadd('json-path-with-types', '#/curation-export/manifest/#int/modality')
        # addr_jprai = address_from_fadd_type_fadd('json-path-with-types', '#/curation-export/samples/#int/raw_anat_index')
        # addr_jpnai = address_from_fadd_type_fadd('json-path-with-types', '#/curation-export/samples/#int/norm_anat_index')
        # addr_jpsuid = address_from_fadd_type_fadd('json-path-with-types', '#/curation-export/subjects/#int/id_sub')
        # addr_jpsaid = address_from_fadd_type_fadd('json-path-with-types', '#/curation-export/samples/#int/id_sam')

        self.addr_const_null = q.address_from_fadd_type_fadd("constant", None)

        # qd_rai = desc_quant_from_label('reva ft sample anatomical location distance index raw')
        self.qd_nai = q.desc_quant_from_label(
            "reva ft sample anatomical location distance index normalized v1"
        )
        self.qd_nain = q.desc_quant_from_label(
            "reva ft sample anatomical location distance index normalized v1 min"
        )
        self.qd_naix = q.desc_quant_from_label(
            "reva ft sample anatomical location distance index normalized v1 max"
        )

        self.cd_mod = q.desc_cat_from_label_domain_label(
            "hasDataAboutItModality", None
        )
        self.cd_bot = q.desc_cat_from_label_domain_label(
            "bottom", None
        )  # we just need something we can reference that points to null so we can have a refernce to all the objects

        self.id_human = q.desc_inst_from_label("human")
        self.id_nerve = q.desc_inst_from_label("nerve")
        self.id_nerve_volume = q.desc_inst_from_label("nerve-volume")
        self.luid = {
            "human": self.id_human,
            "nerve": self.id_nerve,
            "nerve-volume": self.id_nerve_volume,
        }

        self.ct_mod = q.cterm_from_label("microct")  # lol ct ct
        self.ct_hack = q.cterm_from_label("hack-associate-some-value")
        self.luct = {
            "ct-hack": self.ct_hack,
            "microct": self.ct_mod,
        }

In [21]:
Units("mm", "http://qudt.org/vocab/unit/MM")

Units(label='mm', iri='http://qudt.org/vocab/unit/MM')

In [71]:
from dataclasses import dataclass, asdict
import typing
from typing import ClassVar, Dict, Protocol, Any

type UUID = str


class Dataclass(Protocol):
    __dataclass_fields__: ClassVar[Dict[str, Any]]


@dataclass
class Unit:
    __table__ = "units"
    label: str
    iri: str


@dataclass
class Objects:
    __table__ = "objects"
    id: UUID
    id_type: str
    id_file: int | None = None
    id_internal: UUID | None = None


@dataclass
class DescriptorQuant:
    __table__ = "descriptors_quant"
    label: str
    iri: str


class Ingest:

    def __init__(self, session: Session):
        self.session = session

    def execute_insert(
        self, query: str, dataclasses: typing.Sequence[Dataclass]
    ) -> None:
        """Execute a query with dataclasses as parameters

        Parameters
        ----------
        query : str
            sql query string
        dataclasses : typing.Sequence[Dataclass]
            a list of SQL table dataclasses to be inserted
        """
        statement = sql_text(query)
        params = [asdict(dataclass) for dataclass in dataclasses]
        try:
            self.session.execute(statement=statement, params=params)
            self.session.commit()
        except Exception as e:
            print(f"Error: {e}")
            self.session.rollback()

    def insert_units(self, units: typing.Sequence[Unit]) -> None:
        query = r"INSERT INTO units (label, iri) VALUES (:label, :iri)"
        self.execute_insert(query=query, dataclasses=units)

    def insert_objects(self, objects: typing.Sequence[Objects]) -> None:
        query = r"INSERT INTO objects (id, id_type) VALUES (:id, :id_type) ON CONFLICT DO NOTHING"
        self.execute_insert(query=query, dataclasses=objects)

    def insert_descriptors_quant(
        self, descriptors: typing.Sequence[DescriptorQuant]
    ) -> None:
        query = (
            r"INSERT INTO descriptors_quant (label, iri) VALUES (:label, :iri)"
        )
        self.execute_insert(query=query, dataclasses=descriptors)


ingest = Ingest(session=session)

ingest.insert_units(
    units=[
        Unit(
            label="pixel",
            iri="http://uri.interlex.org/tgbugs/uris/readable/aspect/unit/pixel",
        ),
    ]
)

ingest.insert_objects(
    objects=[
        Objects(id=dataset["content"]["id"].split(":")[-1], id_type="dataset"),
    ]
)

2024-09-12 07:32:03,543 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 07:32:03,544 INFO sqlalchemy.engine.Engine INSERT INTO units (label, iri) VALUES (%(label)s, %(iri)s)
2024-09-12 07:32:03,544 INFO sqlalchemy.engine.Engine [cached since 5593s ago] {'label': 'pixel', 'iri': 'http://uri.interlex.org/tgbugs/uris/readable/aspect/unit/pixel'}
Error: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "units_label_key"
DETAIL:  Key (label)=(pixel) already exists.

[SQL: INSERT INTO units (label, iri) VALUES (%(label)s, %(iri)s)]
[parameters: {'label': 'pixel', 'iri': 'http://uri.interlex.org/tgbugs/uris/readable/aspect/unit/pixel'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
2024-09-12 07:32:03,636 INFO sqlalchemy.engine.Engine ROLLBACK
2024-09-12 07:32:03,681 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-12 07:32:03,681 INFO sqlalchemy.engine.Engine INSERT INTO objects (id, id_type) VALUES (%(id)s, %(id_type)s) ON CONFLI

In [66]:
dataset["content"]["id"]

'N:dataset:aa43eda8-b29a-4c25-9840-ecbd57598afc'

In [54]:
# wrap columns
q = Queries(session)
i = InternalIds(q)

# updated_transitive,
# values_objects,
# values_dataset_object,
# make_values_instances,
# make_values_parents,
# make_void,
# make_vocd,
# make_voqd,
# make_values_cat,
# make_values_quant,


class Ingests:
    """Create ingest objects for a dataset using Pennsieive direct"""

    def __init__(self, dataset: str) -> None:
        self.dataset = dataset

    def extract(self):
        return (self.updated_transitive(), self.values_objects())

    def updated_transitive(self):
        """Last updated Timestamp"""
        return max(
            [v["content"]["updatedAt"] for v in self.dataset["children"]]
        )
    
    def values_objects(self):
        
        
Ingests(dataset).extract()

IndentationError: expected an indented block after function definition on line 32 (4174477168.py, line 36)

In [60]:
from __future__ import annotations
from functools import cache
from pathlib import Path
from pydantic import BaseSettings, BaseModel
import boto3
from botocore.client import BaseClient
import requests
from typing import Tuple, List, Any


class PennsieveModel(BaseSettings):
    PENNSIEVE_API_TOKEN: str
    PENNSIEVE_API_SECRET: str
    auth_url: str = "https://api.pennsieve.io/authentication/cognito-config"
    private_dataset_url: str = "https://api.pennsieve.io/datasets/"
    public_dataset_url: str = "https://api.pennsieve.io/discover/datasets"

    class Config:  # type: ignore

        env_file = Path.home() / ".scicrunch/credentials/pennsieve"
        fields = {
            "PENNSIEVE_API_TOKEN": {"env": "PENNSIEVE_API_TOKEN"},
            "PENNSIEVE_API_SECRET": {"env": "PENNSIEVE_API_SECRET"},
        }


class Settings(BaseSettings):
    pennsieve: PennsieveModel = PennsieveModel()


class PennsieveClient:
    def __init__(self):
        settings = Settings().pennsieve
        self.public_dataset_url = settings.public_dataset_url
        self.private_dataset_url = settings.private_dataset_url
        self.auth_url = settings.auth_url
        self.api_key = self.__aws_bot_login(
            token=settings.PENNSIEVE_API_TOKEN,
            secret=settings.PENNSIEVE_API_SECRET,
        )
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "accept": "*/*",
        }
        self.session = requests.Session()

    def __reauth(self) -> None:
        settings = Settings().pennsieve
        self.api_key = self.__aws_bot_login(
            token=settings.PENNSIEVE_API_TOKEN,
            secret=settings.PENNSIEVE_API_SECRET,
        )
        self.headers = {
            "Authorization": f"Bearer {self.api_key}",
            "accept": "*/*",
        }

    def __aws_bot_login(self, token: str, secret: str) -> str:
        """Awful AWS bot login for real API key using token + secrets"""
        r = requests.get(self.auth_url)
        r.raise_for_status()
        cognito_app_client_id = r.json()["tokenPool"]["appClientId"]
        cognito_region = r.json()["region"]
        cognito_idp_client: BaseClient = boto3.client(  # type: ignore
            "cognito-idp",
            region_name=cognito_region,
            aws_access_key_id="",
            aws_secret_access_key="",
        )
        login_response = cognito_idp_client.initiate_auth(  # type: ignore
            AuthFlow="USER_PASSWORD_AUTH",
            AuthParameters={"USERNAME": token, "PASSWORD": secret},
            ClientId=cognito_app_client_id,
        )
        api_key: str = login_response["AuthenticationResult"]["AccessToken"]
        return api_key

    def __get(
        self,
        url: str,
        params: dict[str, Any] | None = None,
    ) -> requests.Response:
        """
        Get request to endpoint with params

        Parameters
        ----------
        endpoint : URL
            API endpoint to hit
        params : dict, optional
            API options, by default None

        Returns
        -------
        dict
            API json response
        """
        response = requests.get(url, headers=self.headers, params=params)
        response.raise_for_status()
        return response

    def _post(
        self,
        url: str,
        data: dict[str, Any] | None = None,
        payload: dict[str, Any] | None = None,
    ) -> requests.Response:
        """
        Post request to endpoint with data or json payload

        Parameters
        ----------
        endpoint : URL
            API endpoint to hit
        data : dict, optional
            API contents, by default None
        json : dict, optional
            API contents, by default None

        Returns
        -------
        dict
            API json response
        """
        response = self.session.post(
            url, data=data, json=payload, headers=self.headers
        )
        # Pennsieve API is unstable, only fail on 500s
        if response.status_code >= 400:
            print(response.text)
            self.__reauth()
            response = self.session.post(
                url, data=data, json=payload, headers=self.headers
            )
        response.raise_for_status()
        return response

    @staticmethod
    def create_path(path: Path) -> None:
        try:
            path.mkdir(parents=True, exist_ok=False)
        except FileExistsError:
            pass

    def get_user(self):
        return self.__get("https://api.pennsieve.io/user/").json()

    @cache
    def get_dataset(self, id_or_name: str) -> dict[str, Any]:
        return self.__get(
            f"https://api.pennsieve.io/datasets/{id_or_name}"
        ).json()

    def get_dataset_packages(self, id_or_name: str) -> dict[str, Any]:
        return self.__get(
            f"https://api.pennsieve.io/datasets/{id_or_name}/packages"
        ).json()["packages"]

    @cache
    def get_dataset_manifest(self, id_or_name: str) -> dict[str, Any]:
        url = "https://api.pennsieve.io/packages/download-manifest"
        dataset = self.get_dataset(id_or_name)
        payload = {
            "nodeIds": [
                child["content"]["nodeId"] for child in dataset["children"]
            ]
        }
        dataset["files"] = self._post(url, payload=payload).json()
        return dataset

    @cache
    def get_child_manifest(self, node_id: str) -> dict[str, Any]:
        url = "https://api.pennsieve.io/packages/download-manifest"
        payload = {"nodeIds": [node_id]}
        resp = self._post(url, payload=payload)
        return resp.json()

    def export_dataset(
        self, id_or_name: str, output_dir: Path | str, verbose: bool = True
    ) -> None:
        """Export a dataset to a directory

        Parameters
        ----------
        id_or_name : str
            Dataset ID or name
        output_dir : Path | str
            Output directory
        verbose : bool, optional
            Prints filename being downloaded, by default True
        """
        url = "https://api.pennsieve.io/packages/download-manifest"
        output_dir = Path(output_dir) / id_or_name
        # Pull dataset for root children IDs
        dataset = self.get_dataset(id_or_name)
        for child in dataset["children"]:
            payload = {"nodeIds": [child["content"]["nodeId"]]}
            # Pull tree for 1 child at a time since prebuilt s3 links only last a couple hours
            # If all children are pulled at once, the links will expire before all files are downloaded if over 200GBs
            manifest = self._post(url, payload=payload).json()
            for filemeta in manifest["data"]:
                parents_path = output_dir / "/".join(filemeta["path"])
                # Create nested parent directories if they don"t exist
                self.create_path(parents_path)
                file_path = parents_path / filemeta["fileName"]
                if verbose:
                    print(f"downloading path: {file_path}")
                url = filemeta["url"]
                # If file already exists, skip if it is stopped in the middle of downloading
                # TODO: query file for checksum; very slow but will garuntee no partial downloads
                if file_path.exists():
                    continue
                # Stream download to file
                with requests.get(url, stream=True) as r:
                    r.raise_for_status()
                    with open(file_path, "wb") as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)

    def _private_datasets(self):
        """Get Private dataset for it"s N:# ID

        Notes
        -----
        There is no known limiter at the moment. Expect all results and be conservative.
        """
        return self.__get(self.private_dataset_url).json()

    def _public_datasets(
        self, limit: int = 1, offset: int = 0, asc: bool = False
    ) -> list[dict[str, Any]]:
        """Everything about the dataset except the N:# id itself"""
        params = {
            "limit": limit,
            "offset": offset,
            "orderBy": "date",
            "orderDirection": "asc" if asc else "desc",
        }
        return self.__get(self.public_dataset_url, params=params).json()[
            "datasets"
        ]

    def get_datasets(self) -> List[dict[str, Any]]:
        """
        Get all complete datasets
        """
        # Private API endpoint Needs to be normalized via content key
        intId_id = {
            d["content"]["intId"]: d["content"]["id"]
            for d in self._private_datasets()
        }
        datasets = self._public_datasets(limit=100000)
        for dataset in datasets:
            dataset["dataset_id"] = intId_id.get(dataset["sourceDatasetId"])
        return datasets

    def get_partial_hash(self) -> Tuple[int, int, int | None]:
        """Get latest dataset id & version to see if anything new from pennsieve is worth pulling"""
        latest_dataset = self._public_datasets()[0]
        partial_hash = (
            latest_dataset["sourceDatasetId"],
            latest_dataset["version"],
            latest_dataset["revision"],
        )
        return partial_hash